In [2]:
import os
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import pandas as pd
import skimage
from skimage.io import imread,imshow, imsave
from skimage.transform import resize
import matplotlib.pyplot as plt
import sys
from sklearn.model_selection import train_test_split

In [3]:
# users need to just select dimension and channel
dimension = 256
channel = 3

In [4]:
DATA_DIR = '../data/'
REL_IMAGE_PATH = os.path.join(DATA_DIR, "images-"+str(dimension))
REL_MASK_PATH = os.path.join(DATA_DIR, "masks-"+str(dimension))

IMAGE_TRAIN_PATH  = os.path.join(REL_IMAGE_PATH, "train")
IMAGE_TEST_PATH  = os.path.join(REL_IMAGE_PATH, "test")

MASK_TRAIN_PATH = os.path.join(REL_MASK_PATH, "train")
MASK_TEST_PATH = os.path.join(REL_MASK_PATH, "test")

In [5]:
train_image_names = os.listdir(IMAGE_TRAIN_PATH)
test_image_names = os.listdir(IMAGE_TEST_PATH)

train_mask_names = os.listdir(MASK_TRAIN_PATH)
test_mask_names = os.listdir(MASK_TEST_PATH)

In [6]:
width = dimension
height = dimension
# #to create X and Y arrays to be later filled with images of both images and masks
x_train = np.zeros((len(train_image_names),height,width,channel),dtype = np.uint8)
y_train = np.zeros((len(train_mask_names),height,width,1),dtype = np.bool)

In [7]:
if os.path.isdir(IMAGE_TRAIN_PATH):
    i = 0
    for image_name in train_image_names:
        if i == 10:
            continue
        image_path = os.path.join(IMAGE_TRAIN_PATH, image_name)
        img = imread(image_path)[:,:,:channel]
        x_train[i] = resize(img,(height,width),mode ='edge',preserve_range =True)
        
        mask_path = os.path.join(MASK_TRAIN_PATH, image_name[:-4]+"_mask.png")
        mask = imread(mask_path)
        mask_resized = resize(mask, (height,width), mode ='edge', preserve_range = True)
        mask = np.expand_dims(mask_resized, axis=-1)
        y_train[i] = mask
        
        i += 1
else:
    print("There is no folder: "+img_output_loc+". Please build the data-set using BuildNewDataSet notebook.")

In [7]:
x_train_1 = x_train[:10]
y_train_1 = y_train[:10]

In [8]:
def jaccard_distance(y_true, y_pred, smooth=100):
    #""" Calculates mean of Jaccard distance as a loss function """
    intersection = tf.reduce_sum(y_true * y_pred, axis=(1,2))
    sum_ = tf.reduce_sum(y_true + y_pred, axis=(1,2))
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    jd =  (1 - jac) * smooth
    return tf.reduce_mean(jd)

In [9]:
#creating U-Net architecture
def create_model(width, height, channel, loss, metric):
    inputs = tf.keras.layers.Input((width,height,channel))
    s = tf.keras.layers.Lambda(lambda x : x/255)(inputs)
    c1 = tf.keras.layers.Conv2D(16,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(s)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.Conv2D(16,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2,2))(c1)

    c2 = tf.keras.layers.Conv2D(32,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(p1)
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.Conv2D(32,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2,2))(c2)

    c3 = tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(p2)
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2,2))(c3)

    c4 = tf.keras.layers.Conv2D(128,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(p3)
    c4 = tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.Conv2D(128,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(c4)
    p4 = tf.keras.layers.MaxPooling2D((2,2))(c4)


    c5 = tf.keras.layers.Conv2D(256,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(p4)
    c5 = tf.keras.layers.Dropout(0.3)(c5)
    c5 = tf.keras.layers.Conv2D(256,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(c5)


    u6 = tf.keras.layers.Conv2DTranspose(128,(2,2),strides = (2,2),padding = 'same')(c5)
    u6 = tf.keras.layers.concatenate([u6,c4])
    c6 = tf.keras.layers.Conv2D(128,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(u6)
    c6 = tf.keras.layers.Dropout(0.3)(c6)
    c6 = tf.keras.layers.Conv2D(128,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(64,(2,2),strides = (2,2),padding = 'same')(c6)
    u7 = tf.keras.layers.concatenate([u7,c3])
    c7 = tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(u7)
    c7 = tf.keras.layers.Dropout(0.2)(c7)
    c7 = tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(32,(2,2),strides = (2,2),padding = 'same')(c7)
    u8 = tf.keras.layers.concatenate([u8,c2])
    c8 = tf.keras.layers.Conv2D(32,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(u8)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(16,(2,2),strides = (2,2),padding = 'same')(c8)
    u9 = tf.keras.layers.concatenate([u9,c1],axis = 3)
    c9 = tf.keras.layers.Conv2D(16,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(u9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(c9)

    outputs = tf.keras.layers.Conv2D(1,(1,1),activation = "sigmoid")(c9)

    model = tf.keras.Model(inputs = [inputs],outputs = [outputs])
    model.compile(optimizer = 'adam',loss = loss,metrics = [metric])
    #model.summary()
    return model

In [8]:
#creating U-Net architecture
inputs = tf.keras.layers.Input((width,height,channel))
s = tf.keras.layers.Lambda(lambda x : x/255)(inputs)
c1 = tf.keras.layers.Conv2D(16,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2,2))(c1)

c2 = tf.keras.layers.Conv2D(32,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2,2))(c2)

c3 = tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2,2))(c3)

c4 = tf.keras.layers.Conv2D(128,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(c4)
p4 = tf.keras.layers.MaxPooling2D((2,2))(c4)


c5 = tf.keras.layers.Conv2D(256,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(c5)


u6 = tf.keras.layers.Conv2DTranspose(128,(2,2),strides = (2,2),padding = 'same')(c5)
u6 = tf.keras.layers.concatenate([u6,c4])
c6 = tf.keras.layers.Conv2D(128,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(u6)
c6 = tf.keras.layers.Dropout(0.3)(c6)
c6 = tf.keras.layers.Conv2D(128,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(c6)

u7 = tf.keras.layers.Conv2DTranspose(64,(2,2),strides = (2,2),padding = 'same')(c6)
u7 = tf.keras.layers.concatenate([u7,c3])
c7 = tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(c7)

u8 = tf.keras.layers.Conv2DTranspose(32,(2,2),strides = (2,2),padding = 'same')(c7)
u8 = tf.keras.layers.concatenate([u8,c2])
c8 = tf.keras.layers.Conv2D(32,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(c8)

u9 = tf.keras.layers.Conv2DTranspose(16,(2,2),strides = (2,2),padding = 'same')(c8)
u9 = tf.keras.layers.concatenate([u9,c1],axis = 3)
c9 = tf.keras.layers.Conv2D(16,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16,(3,3),activation = 'relu',kernel_initializer='he_normal',padding = 'same')(c9)

outputs = tf.keras.layers.Conv2D(1,(1,1),activation = "sigmoid")(c9)

model = tf.keras.Model(inputs = [inputs],outputs = [outputs])
model.compile(optimizer = 'adam',loss = ['binary_crossentropy'],metrics = ['accuracy'])
model.summary()



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 256, 256, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 448         lambda[0][0]                     
________________________________________________________________________________

In [11]:
#model = create_model(width, height, channel, 'cosine_similarity', 'accuracy')

In [9]:
model_path="cosine_similarity_256x256-{epoch:02d}-{val_acc:.2f}.hdf5"
clbk = [tf.keras.callbacks.ModelCheckpoint(model_path, verbose= 1,save_best_only=True),
        tf.keras.callbacks.EarlyStopping(patience =2, monitor = 'val_acc', mode = 'min'),
        tf.keras.callbacks.TensorBoard(log_dir = "logs")]

results = model.fit(x_train[:10],y_train[:10], epochs=20,
                   verbose = 1,use_multiprocessing=True,
                   callbacks=clbk, validation_split=0.2)

Train on 8 samples, validate on 2 samples
Epoch 1/20

Epoch 00001: val_loss improved from inf to 0.80551, saving model to cosine_similarity_256x256-01-0.01.hdf5
8/8 [==============================] - 4s 521ms/sample - loss: 0.8171 - acc: 0.2103 - val_loss: 0.8055 - val_acc: 0.0063
Epoch 2/20

Epoch 00002: val_loss improved from 0.80551 to 0.71331, saving model to cosine_similarity_256x256-02-0.37.hdf5
8/8 [==============================] - 3s 325ms/sample - loss: 0.7408 - acc: 0.2678 - val_loss: 0.7133 - val_acc: 0.3722
Epoch 3/20

Epoch 00003: val_loss improved from 0.71331 to 0.66204, saving model to cosine_similarity_256x256-03-0.76.hdf5
8/8 [==============================] - 3s 350ms/sample - loss: 0.7031 - acc: 0.4926 - val_loss: 0.6620 - val_acc: 0.7589


In [ ]:
90.6585 
90.6543 
90.6679
90.6838
90.6768
90.6782
90.6067